<a href="https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Notebooks/01-proceso-analisis-exploratorio/EJERCICIO_2__proceso_analisis_exploratorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EJERCICIO 2: Proceso Análisis Exploratorio 
Ejercicio para practicar la introducción al EDA (Exploratory Data Analysis).

Notebook por [Javier Blanco Cordero](https://www.linkedin.com/in/javier-blanco-cordero-71373656/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/1IT6OVy9YS_sbgHF2oU9D0L--fZiyUfij7NxVhu3cAdg/edit?usp=sharing)
*   [Enlace a notebook con introducción al EDA](https://colab.research.google.com/drive/1B37GMOCjKQ0S8NoRI6lALEW9EZtE0Hit#scrollTo=R53FGLN8jo1L)
*   [Enlace a este notebook en Colab](https://colab.research.google.com/drive/1VMkJls178kiMWqEBRXvFxw47UWMcivfm)




# Objetivo
Eres un inversor privado que está considerando el sp500 como producto. Basándote en el histórico del sp500, resuelve las preguntas que puedes ver a continuación.


Primero, claro, debes importar el dataset del sp500 y las librerías necesarias.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Generamos un objeto para subir el archivo
from google.colab import files
uploaded = files.upload()

Saving sp500.csv to sp500 (1).csv


In [3]:
# Convertimos los bytes a formato tabular
import io
buffer_io = io.BytesIO(uploaded['sp500.csv'])


# Cargamos en un dataframe
df_sp500 = pd.read_csv(buffer_io)
df_sp500.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0


# Pregunta 1
### Cuánto valdría mi inversión ahora si hubiera metido 1 dollar el primer día (1927-12-30)
Nota: ese dollar es equivalente a unos $15.22 de ahora por la inflacción. Ha superado el sp500 la inflacción?

In [4]:
# Variación respecto a ayer
df_sp500['Variacion'] = df_sp500['Close']/df_sp500['Close'].shift(1)

In [5]:
# Solución 1: Producto de la variación diaria
np.array(df_sp500['Variacion'].dropna()).prod()

194.98527412231164

In [6]:
# Solución 2: Variación entre el primer y el último registro
df_sp500.iloc[-1,5]/df_sp500.iloc[0,5]

194.98527412231027

# Pregunta 2
### Qué porcentaje de días el sp500 ha acabado por encima del día valor de cierre del día anterior

In [7]:
# Filtro que determina qué días se ha crecido
filtro_verde = df_sp500['Variacion'] > 1
filtro_verde[:5]

0    False
1     True
2    False
3    False
4     True
Name: Variacion, dtype: bool

In [8]:
# Porcentaje que supone sobre el total
filtro_verde.sum()/len(df_sp500)
# Ok, pues se gana un 52.3% de los días (se pierde bastantes días)

0.5236890622990181

# Pregunta 3
Considerando un año como los ultímos 365 días, es decir, los últimos 250 días de bolsa (cierra findes y festivos), 
### Cuál es la ganancia media anual esperable


In [9]:
# Calculamos primero la variación en los últimos 250 registros para cada registro disponible
df_sp500['Variacion ac ult 250'] = df_sp500['Variacion'].rolling(250).agg(lambda x:x.prod())

In [10]:
# Describe nos da no solo la media, sino también otros estadísticos relevantes
df_sp500.describe()

,Open,High,Low,Close,Adj Close,Volume,Variacion,Variacion ac ult 250
count,23323.000000,23323.000000,23323.000000,23323.000000,23323.000000,2.332300e+04,23322.000000,23073.000000
mean,486.821737,489.690119,483.762812,486.918598,486.918598,7.691668e+08,1.000298,1.076378
std,726.187716,730.112168,721.890344,726.278798,726.278798,1.484864e+09,0.012000,0.200249
min,4.400000,4.400000,4.400000,4.400000,4.400000,0.000000e+00,0.795331,0.294235
25%,23.855001,23.855001,23.855001,23.855001,23.855001,1.280000e+06,0.995466,0.960681
50%,99.500000,100.309998,98.720001,99.500000,99.500000,1.734000e+07,1.000482,1.088715
75%,834.035004,843.744995,822.889984,834.704987,834.704987,5.378550e+08,1.005416,1.195957
max,3564.739990,3588.110107,3535.229980,3580.840088,3580.840088,1.145623e+10,1.166096,2.741573


# Pregunta 4
Mi objetivo es invertir una cantidad en el sp500 y extraer mi dinero en 5 años (de nuevo nos referimos a la idea de año descrita en la pregunta anterior). Pero soy tremendamente gafe e invierto el peor día posible de la historia para conseguir mi objetivo.
### Qué día he invertido? Cuánto he perdido al cabo de 5 años? 

In [11]:
250*5

1250

In [12]:
# Solución 1: 
# Creación variable variacion tras 5 años utilizando [::-1] para revertir orden
df_sp500['Variacion ac tras 1250'] = df_sp500[::-1]['Variacion'].rolling(1250).agg(lambda x:x.prod())[::-1]

# Filtro que determina el peor día de compra (respecto a una venta en 5 años)
filtro_gafe = df_sp500['Variacion ac tras 1250'] == df_sp500['Variacion ac tras 1250'].min()

# Visualizamos ese día de compra
df_sp500.loc[filtro_gafe,['Date','Close','Variacion ac tras 1250']]

,Date,Close,Variacion ac tras 1250
423,1929-09-09,31.67,0.266101


In [13]:
# Solución 2: 
# Creación variable variacion ult 5 años
df_sp500['Variacion ac ult 1250'] = df_sp500['Variacion'].rolling(1250).agg(lambda x:x.prod())

# Filtro que determina el peor día de venta (respecto a una compra hace 5 años)
filtro_gafe = df_sp500['Variacion ac ult 1250'] == df_sp500['Variacion ac ult 1250'].min()

# Mostramos la fecha y variacion acumuladas en los últimos 1250 ese día
display('Venta: ', df_sp500.loc[filtro_gafe,['Date','Close','Variacion ac ult 1250']])

# Nos guardamos el valor del índice de ese día
indice_salida = df_sp500.loc[filtro_gafe].index

# Ahora le restamos 1250 (-1) para acceder al valor del índice de hace 5 años
indice_entrada = indice_salida-1250-1

# Utilizamos ese índice para ver el valor de compra
display('Compra ', df_sp500.loc[indice_entrada,['Date','Close','Variacion ac ult 1250']])

'Venta: '

,Date,Close,Variacion ac ult 1250
1672,1934-09-14,8.47,0.266101


'Compra '

,Date,Close,Variacion ac ult 1250
421,1929-09-05,31.02,NaN


In [14]:
# 
print("Invertí el día 1934-09-14.")
print('5 años después había perdido el ' + str((1-0.2661)*100) + '%')

Invertí el día 1934-09-14.
5 años después había perdido el 73.39%


# Pregunta 5
La respuesta a la pregunta anterior se ve influenciada por el crash del 1929. A partir de ahora limitemos nuestro análisis a los últimos del 1955 en adelante.
### Cuál es la distribución de la ganancia a [1,2,5,10,15,20] años?
Nota: con distribución nos referimos a los estadísticos principales: media, mediana, cuantiles principales, etc.

In [15]:
# Primero creamos las variables deseadas para los intervalos pedidos
for n in [1,2,5,10,15,20]:
  
  # Variación en los últimos 250*n registros para cada registro disponible
  df_sp500['Variacion ac ult '+str(n)+' años'] = df_sp500['Variacion'].rolling(250*n).agg(lambda x:x.prod())

In [20]:
# Generamos un filtro que nos seleccione solo las fechas desde 1950
filtro_post_1955 = df_sp500['Date'] >= '1955-01-01'
df_sp500.loc[filtro_post_1955,:].head()

,Date,Open,High,Low,Close,Adj Close,Volume,Variacion,Variacion ac ult 250,Variacion ac tras 1250,Variacion ac ult 1250,Variacion ac ult 1 años,Variacion ac ult 2 años,Variacion ac ult 5 años,Variacion ac ult 10 años,Variacion ac ult 15 años,Variacion ac ult 20 años
6747,1955-01-03,36.750000,36.750000,36.750000,36.750000,36.750000,4570000,1.021401,1.461814,1.638966,2.181009,1.461814,1.393629,2.181009,2.805344,2.877839,3.836117
6748,1955-01-04,36.419998,36.419998,36.419998,36.419998,36.419998,4420000,0.991020,1.453312,1.601633,2.151211,1.453312,1.383213,2.151211,2.742470,2.860958,3.845829
6749,1955-01-05,35.520000,35.520000,35.520000,35.520000,35.520000,4640000,0.975288,1.424789,1.623833,2.091873,1.424789,1.361963,2.091873,2.664666,2.805687,3.754757
6750,1955-01-06,35.040001,35.040001,35.040001,35.040001,35.040001,5300000,0.986487,1.412903,1.666948,2.051522,1.412903,1.354988,2.051522,2.601336,2.765588,3.711865
6751,1955-01-07,35.330002,35.330002,35.330002,35.330002,35.330002,4030000,1.008276,1.417168,1.687785,2.074574,1.417168,1.357802,2.074574,2.618977,2.815140,3.819460


In [21]:
# Ejecutamos el método describe() sobre el dataframe filtrado
df_sp500.loc[filtro_post_1955,[var for var in df_sp500.columns if 'años' in var]].describe()

,Variacion ac ult 1 años,Variacion ac ult 2 años,Variacion ac ult 5 años,Variacion ac ult 10 años,Variacion ac ult 15 años,Variacion ac ult 20 años
count,16576.000000,16576.000000,16576.000000,16576.000000,16576.000000,16576.000000
mean,1.085090,1.177332,1.496520,2.206940,3.260770,4.588016
std,0.158046,0.235627,0.469407,0.874937,1.609481,2.458483
min,0.512270,0.485248,0.607386,0.520108,1.090909,1.547355
25%,0.989114,1.045031,1.128437,1.537978,1.835525,2.699654
50%,1.096430,1.167769,1.483711,2.151070,3.139417,4.004788
75%,1.187664,1.315280,1.751907,2.849368,4.275529,5.953456
max,1.683148,1.970717,3.192277,5.147127,9.083157,14.675931


In [25]:
# 
df_sp500[['Date']+[var for var in df_sp500.columns if 'años' in var]].tail()

,Date,Variacion ac ult 1 años,Variacion ac ult 2 años,Variacion ac ult 5 años,Variacion ac ult 10 años,Variacion ac ult 15 años,Variacion ac ult 20 años
23318,2020-10-29,1.079298,1.215585,1.595234,2.803444,2.619380,2.433922
23319,2020-10-30,1.062272,1.194153,1.598244,2.728719,2.600635,2.438576
23320,2020-11-02,1.076634,1.201343,1.636270,2.783117,2.635877,2.522760
23321,2020-11-03,1.095028,1.197332,1.640939,2.836566,2.675274,2.547107
23322,2020-11-04,1.116123,1.226807,1.679366,2.916810,2.731956,2.637439
